# Tier 1 Model Pipeline - Free Tier (1 Modality, 5 Days, XGBoost Only)

In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

# Configurations
N_DAYS = 5
N_MODALITIES = 1
IMG_SIZE = (64, 64)
BASE_DIR = "/kaggle/input/data-set-3k-zip/data_set_3K"
MAX_SAMPLES_PER_CLASS = 50


In [ ]:
def load_data(base_dir, max_per_class):
    X, y = [], []
    for label in ["0", "1"]:
        label_dir = os.path.join(base_dir, label)
        count = 0
        for folder in sorted(os.listdir(label_dir)):
            if count >= max_per_class:
                break
            try:
                day_images = []
                for day in range(1, N_DAYS + 1):
                    img_path = os.path.join(label_dir, folder, "1", f"{day:02}.png")
                    img = cv2.imread(img_path)
                    if img is None:
                        raise ValueError("Missing image")
                    img = cv2.resize(img, IMG_SIZE)
                    img = img.astype("float32") / 255.0
                    day_images.append(img)
                sample = np.stack(day_images, axis=0)
                X.append(sample)
                y.append(int(label))
                count += 1
            except:
                continue
    return np.array(X), np.array(y)

X, y = load_data(BASE_DIR, MAX_SAMPLES_PER_CLASS)
print("Loaded:", X.shape, y.shape)


In [ ]:
X_flat = X.reshape(len(X), -1)
X_train, X_test, y_train, y_test = train_test_split(X_flat, y, test_size=0.2, stratify=y, random_state=42)


In [ ]:
xgb = XGBClassifier(n_jobs=1, use_label_encoder=False, eval_metric="logloss")
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)

print(classification_report(y_test, y_pred))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred, display_labels=["Non-Toxic", "Toxic"], cmap="Greens")
plt.title("XGBoost - Tier 1 Model (Free Tier)")
plt.show()
